In [5]:
from PIL import Image
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [6]:
labels = pd.read_csv("Xray_TeethLabels_Simple.csv",index_col=0)
Ylabels = labels[5:]

metaY = labels[:3]

X = {}
Y = {}
count1 = 0
count2 = 0
count3 = 0
for picFilename in glob.glob("processed3/processed/*"):
    patNumber =  picFilename.split('/')[2].split('_')[0]
    toothNumber1 = picFilename.split('/')[2].split('_')[1]
    toothNumber2 = picFilename.split('/')[2].split('_')[2].split('.')[0]
    im = Image.open(picFilename)

    if int(metaY.loc['Age',patNumber]) < 25:
        count1+=1
        if 'Age1' in X.keys():
            X['Age1'][patNumber + '_' + toothNumber1 + '_' + toothNumber2] = np.array(im).flatten()
            Y['Age1'].append(1 if (Ylabels.loc[toothNumber1,patNumber] == 'Yes' or Ylabels.loc[toothNumber2,patNumber] == 'Yes') else 0)
        else:
            X['Age1'] = pd.DataFrame()
            X['Age1'][patNumber + '_' + toothNumber1 + '_' + toothNumber2] = np.array(im).flatten()
            Y['Age1'] = [1 if (Ylabels.loc[toothNumber1,patNumber] == 'Yes' or Ylabels.loc[toothNumber2,patNumber] == 'Yes') else 0]
    
    elif int(metaY.loc['Age',patNumber]) < 40:
        count2+=1
        if 'Age2' in X.keys():
            X['Age2'][patNumber + '_' + toothNumber1 + '_' + toothNumber2] = np.array(im).flatten()
            Y['Age2'].append(1 if (Ylabels.loc[toothNumber1,patNumber] == 'Yes' or Ylabels.loc[toothNumber2,patNumber] == 'Yes') else 0)
        else:
            X['Age2'] = pd.DataFrame()
            X['Age2'][patNumber + '_' + toothNumber1 + '_' + toothNumber2] = np.array(im).flatten()
            Y['Age2'] = [1 if (Ylabels.loc[toothNumber1,patNumber] == 'Yes' or Ylabels.loc[toothNumber2,patNumber] == 'Yes') else 0]
    
    else:
        count3+=1
        if 'Age3' in X.keys():
            X['Age3'][patNumber + '_' + toothNumber1 + '_' + toothNumber2] = np.array(im).flatten()
            Y['Age3'].append(1 if (Ylabels.loc[toothNumber1,patNumber] == 'Yes' or Ylabels.loc[toothNumber2,patNumber] == 'Yes') else 0)
        else:
            X['Age3'] = pd.DataFrame()
            X['Age3'][patNumber + '_' + toothNumber1 + '_' + toothNumber2] = np.array(im).flatten()
            Y['Age3'] = [1 if (Ylabels.loc[toothNumber1,patNumber] == 'Yes' or Ylabels.loc[toothNumber2,patNumber] == 'Yes') else 0]

In [7]:
np.random.seed(100)
def svc_param_selection(X, y, nfolds):
    C = [0.1, 1, 10, 100, 1000] #try different values for C
    kernel = ['linear', 'rbf']
    param_grid = {'C': C, 'kernel': kernel}
    grid_search = GridSearchCV(SVC(), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [8]:
for age in X.keys():
    ageX = X[age]
    ageY = Y[age]

    X_train, X_test, y_train, y_test = train_test_split(ageX.T.to_numpy(), np.array(ageY), test_size=0.20, random_state=100)
    
    best_c = svc_param_selection(ageX.T.to_numpy(), np.array(ageY), 2)
    print(f"SVM modle for age: {age}")
    print ("The best value for C is:",best_c)
    model = SVC(C=best_c['C'], kernel=best_c['kernel'])
    model.fit(X_train, y_train)
    print(model)
    # make predictions
    expected = y_test
    predicted = model.predict(X_test)
    # summarize the fit of the model
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))
    tn, fp, fn, tp = metrics.confusion_matrix(y_test, predicted).ravel()
    print(tp,tn,fp,fn)
    print(f"sensitivity: {tp/(tp+fn)}")
    print(f"Specificity: {tn/(tn+fp)}")
    print(f"FPR: {fp/(fp+tn)}")
    print(f" FNR: {fn/(fn+tp)}")
    print(f"Accuracy: {(tp+tn)/(tp+tn+fp+fn)}")
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    print(f"precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 score: {(2*precision*recall)/(precision+recall)}")


SVM modle for age: Age2
The best value for C is: {'C': 0.1, 'kernel': 'rbf'}
SVC(C=0.1)


/usr/local/anaconda3/envs/opencv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/envs/opencv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/envs/opencv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.80      1.00      0.89         8

    accuracy                           0.80        10
   macro avg       0.40      0.50      0.44        10
weighted avg       0.64      0.80      0.71        10

[[0 2]
 [0 8]]
8 0 2 0
sensitivity: 1.0
Specificity: 0.0
FPR: 1.0
 FNR: 0.0
Accuracy: 0.8
precision: 0.8
Recall: 1.0
F1 score: 0.888888888888889
SVM modle for age: Age3
The best value for C is: {'C': 10, 'kernel': 'rbf'}
SVC(C=10)
              precision    recall  f1-score   support

           0       0.50      0.43      0.46         7
           1       0.43      0.50      0.46         6

    accuracy                           0.46        13
   macro avg       0.46      0.46      0.46        13
weighted avg       0.47      0.46      0.46        13

[[3 4]
 [3 3]]
3 3 4 3
sensitivity: 0.5
Specificity: 0.42857142857142855
FPR: 0.5714285714285714
 FNR: 0.5
Accuracy

/usr/local/anaconda3/envs/opencv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/envs/opencv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/envs/opencv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
